In [ ]:
# Algorithm: Adaptation recommender 3
# Input: partial_trace
# Output: recommended adaptation action

# def recommend_adaptation_3(partial_trace):
#   recommended_adaptation_action = None
#   event_logs = get_past_event_logs()

#   classifier_model = learn_model_for_adaptations(event_logs)
#   trace_class = classifier_model.classify(patrial_trace)

#   recommended_adaptation_action = trace_class.adaptation_action
  
#   return recommended_adaptation_action

In [ ]:
def classify_trace(classifier_model, running_trace):
    return trace_class

def recommend_adaptation(running_trace, classifier_model):

  recommended_adaptation_action = None

  trace_class = classify_trace(classifier_model, running_trace)

  recommended_adaptation_action = trace_class
  
  return recommended_adaptation_action

In [ ]:
def learn_model_for_adaptations(train_event_logs, output_model_name):
    return

def get_expected_adaptation_action_from_event_logs(running_trace, event_logs):
    return

def load_learned_model_for_adaptations(learned_model_name):
    return classifier_model

def evaluate_trace(running_trace, recommended_adaptation_action, event_logs):
    expected_adaptation_action = get_expected_adaptation_action_from_event_logs(running_trace, event_logs)
    if expected_adaptation_action == recommended_adaptation_action:
        return 1
    else:
        return 0
    
def evaluate_test_traces(test_traces_event_logs, learned_model_name, event_logs):
    classifier_model = load_learned_model_for_adaptations(learned_model_name)
    num_traces = len(test_traces_event_logs)
    recommendation_hit_count = 0

    for running_trace in test_traces_event_logs:
        recommended_adaptation_action = recommend_adaptation(running_trace, classifier_model)
        evaluation_result = evaluate_trace(running_trace, recommended_adaptation_action, event_logs)
        if evaluation_result == 1:
            recommendation_hit_count += 1
    
    overall_recommendation_hit_ratio = recommendation_hit_count / num_traces
    return overall_recommendation_hit_ratio

def make_test_traces_event_logs(traces_event_logs):
    # remove after running point in the trace
    return test_traces_event_logs


split_points = [0.2, 0.4, 0.66]
train_test_ratio = 2.0

